In [207]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from xgboost import XGBClassifier, plot_importance

In [208]:
data_files_list = ['result_bus_20240226-112104.csv', 'result_passenger_20240226-112405.csv', 'result_taxi_20240226-112648.csv', 'result_truck_20240226-112812.csv']

In [209]:
train_dataset = pd.DataFrame()
test_dataset = pd.DataFrame()

for data_file in data_files_list:
    data = pd.read_csv('analysis_data' + os.sep + data_file)
    train_data, test_data = train_test_split(data, test_size=0.1, random_state=0)
    train_dataset = pd.concat([train_dataset, train_data])
    test_dataset = pd.concat([test_dataset, test_data])
    
train_dataset.drop(columns=['Unnamed: 0'], inplace=True)
test_dataset.drop(columns=['Unnamed: 0'], inplace=True)

data_columns = train_dataset.columns.to_list()[0:]
train_dataset.head()

,file_name,face_pt1_x_pos,face_pt1_y_pos,face_pt2_x_pos,face_pt2_y_pos,left_eye_pt1_x_pos,left_eye_pt1_y_pos,left_eye_pt2_x_pos,left_eye_pt2_y_pos,right_eye_pt1_x_pos,...,target_right_eye_pt1_y_pos,target_right_eye_pt2_x,target_right_eye_pt2_y_pos,exist_mask,exist_glasses,face_visible,left_eye_visible,left_eye_opened,right_eye_visible,right_eye_opened
6581,R_234_60_M_19_M1_G1_C0_03.jpg,211.0,434.0,483.0,910.0,261.0,599.0,304.0,610.0,377.0,...,587.82,425.23,616.64,1,1,1,1,1,1,1
971,R_218_50_M_17_M1_G0_C0_12.jpg,320.0,468.0,566.0,932.0,380.0,635.0,418.0,656.0,487.0,...,627.46,544.69,654.72,1,0,1,1,1,1,1
3776,R_226_30_M_17_M1_G1_C1_17.jpg,284.0,506.0,600.0,942.0,370.0,633.0,396.0,647.0,478.0,...,629.95,523.56,651.35,1,1,1,1,1,1,1
5292,R_231_50_M_09_M1_G0_C0_14.jpg,144.0,447.0,474.0,1013.0,209.0,669.0,228.0,695.0,295.0,...,656.88,369.80,691.17,1,0,1,1,1,1,1
5097,R_230_40_M_19_M1_G1_C0_18.jpg,41.0,582.0,320.0,1028.0,169.0,685.0,199.0,730.0,194.0,...,705.04,240.05,732.24,1,1,1,1,1,1,1


In [210]:
feature_column_names = data_columns[5:17]
target_column_names = [data_columns[38], data_columns[40]]

In [211]:
feature_column_names

['left_eye_pt1_x_pos',
 'left_eye_pt1_y_pos',
 'left_eye_pt2_x_pos',
 'left_eye_pt2_y_pos',
 'right_eye_pt1_x_pos',
 'right_eye_pt1_y_pos',
 'right_eye_pt2_x_pos',
 'right_eye_pt2_y_pos',
 'left_eye_horizontal',
 'left_eye_vertical',
 'right_eye_horizontal',
 'right_eye_vertical']

In [212]:
target_column_names

['left_eye_opened', 'right_eye_opened']

In [213]:
train_feature_dataset = train_dataset[feature_column_names]
train_feature_data = train_feature_dataset.to_numpy()

train_target_dataset = train_dataset[target_column_names]
train_target_data = train_target_dataset.to_numpy().astype(np.uint8)

In [214]:
test_target_dataset = test_dataset[target_column_names]
test_target_data = test_target_dataset.to_numpy().astype(np.uint8)

test_feature_dataset = test_dataset[feature_column_names]
test_feature_data = test_feature_dataset.to_numpy()

In [215]:
model = XGBClassifier()
model.fit(train_feature_data, train_target_data)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [216]:
pred_data = model.predict(test_feature_data)
pred_data = pred_data.astype(np.uint8)
pred_data

array([[1, 1],
       [1, 1],
       [1, 1],
       ...,
       [1, 1],
       [1, 1],
       [1, 1]], dtype=uint8)

In [217]:
confusion_matrix(test_target_data[:, 0], pred_data[:, 0])

array([[  98,  187],
       [  62, 1966]], dtype=int64)

In [218]:
accuracy_score(test_target_data[:, 0], pred_data[:, 0])

0.8923476005188068

In [219]:
precision_score(test_target_data[:, 0], pred_data[:, 0])

0.913144449605202

In [220]:
recall_score(test_target_data[:, 0], pred_data[:, 0])

0.9694280078895463

In [221]:
f1_score(test_target_data[:, 0], pred_data[:, 0])

0.9404448696484095

In [222]:
confusion_matrix(test_target_data[:, 1], pred_data[:, 1])

array([[  94,  166],
       [  41, 2012]], dtype=int64)

In [223]:
accuracy_score(test_target_data[:, 1], pred_data[:, 1])

0.9105058365758755

In [224]:
precision_score(test_target_data[:, 1], pred_data[:, 1])

0.9237832874196511

In [225]:
recall_score(test_target_data[:, 1], pred_data[:, 1])

0.9800292255236239

In [226]:
f1_score(test_target_data[:, 1], pred_data[:, 1])

0.951075395887497